[e1]This is a chatbot example

In [11]:
from transformers.utils import logging
logging.set_verbosity_error()
import torch
print(torch.__version__)

2.2.1


In [12]:
from transformers import pipeline
chatbot=pipeline(task="conversational",model="facebook/blenderbot-400M-distill")


In [13]:
user_message="Hi, I love skiing and I am looking for a good ski resort in the Alps. Can you help me?"


In [14]:
from transformers import Conversation
conversation = Conversation(user_message)
conversation = chatbot(conversation)

conversation.add_message({"role": "user", "content": "What else do you recommend?"})

print(chatbot(conversation))

Conversation id: dc7ffc23-1041-4aab-9c06-8d420fd97fbb
user: Hi, I love skiing and I am looking for a good ski resort in the Alps. Can you help me?
assistant:  Sure, what kind of resort are you looking for? There are so many to choose from.
user: What else do you recommend?
assistant:  Well, there are a lot of resorts in the Rocky Mountains in Colorado.



This is a translator example

In [15]:
from transformers import pipeline
import torch
translator=pipeline(task="translation",model="facebook/nllb-200-distilled-600M",torch_dtype=torch.bfloat16)


In [16]:
text="I am Dogukan Ozkaya. Coming from Turkey. Pleasure to meet you all Love the people in here."
translator(text,src_lang="eng_Latn",tgt_lang="tur_Latn")
print(translator(text,src_lang="eng_Latn",tgt_lang="tur_Latn"))

[{'translation_text': "Ben Dogukan Ozkaya, Türkiye'den geldim."}]


https://github.com/facebookresearch/flores/blob/main/flores200/README.md#languages-in-flores-200
Language abbr

In [17]:
import gc
del chatbot
del translator
gc.collect()


228

In [18]:
summarizer= pipeline(task="summarization",model="facebook/bart-large-cnn",torch_dtype=torch.bfloat16)

In [ ]:
long_text="The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest"
summarizer(long_text,max_length=35,min_length=20)


[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring'}]

: 

: 

In [19]:
del summarizer
gc.collect()

139

In [20]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = ['The cat sits outside',
              'A man is playing guitar',
              'The movies are awesome']

embeddings = model.encode(sentences, convert_to_tensor=True)

print(embeddings)


tensor([[ 0.1392,  0.0030,  0.0470,  ...,  0.0641, -0.0163,  0.0636],
        [ 0.0227, -0.0014, -0.0056,  ..., -0.0225,  0.0846, -0.0283],
        [-0.1043, -0.0628,  0.0093,  ...,  0.0020,  0.0653, -0.0150]],
       device='mps:0')


In [21]:
sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']
embeddings2 = model.encode(sentences2, convert_to_tensor=True)
print(embeddings2)

tensor([[ 0.0163, -0.0700,  0.0384,  ...,  0.0447,  0.0254, -0.0023],
        [ 0.0054, -0.0920,  0.0140,  ...,  0.0167, -0.0086, -0.0424],
        [-0.0842, -0.0592, -0.0010,  ..., -0.0157,  0.0764,  0.0389]],
       device='mps:0')


In [22]:
from sentence_transformers import util

cosine_scores = util.pytorch_cos_sim(embeddings, embeddings2)
print(cosine_scores)
for i in range(len(sentences)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i],
                                                 sentences2[i],
                                                 cosine_scores[i][i]))


tensor([[ 0.2838,  0.1310, -0.0029],
        [ 0.2277, -0.0327, -0.0136],
        [-0.0124, -0.0465,  0.6571]], device='mps:0')
The cat sits outside 		 The dog plays in the garden 		 Score: 0.2838
A man is playing guitar 		 A woman watches TV 		 Score: -0.0327
The movies are awesome 		 The new movie is so great 		 Score: 0.6571


In [23]:
del model
gc.collect()

15

In [25]:
from datasets import load_dataset, load_from_disk

# This dataset is a collection of different sounds of 5 seconds
dataset = load_dataset("ashraq/esc50",
                      split="train[0:10]")
# dataset = load_from_disk("./models/ashraq/esc50/train")

Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 100%|██████████| 2000/2000 [00:06<00:00, 303.13 examples/s]


In [53]:
audio_sample = dataset[2]
display(audio_sample)

{'filename': '1-100210-A-36.wav',
 'fold': 1,
 'target': 36,
 'category': 'vacuum_cleaner',
 'esc10': False,
 'src_file': 100210,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.00668999, -0.01229004, -0.01174699, ..., -0.08509882,
         -0.27288771,  0.        ]),
  'sampling_rate': 48000}}

In [57]:
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample["audio"]["array"],
             rate=audio_sample["audio"]["sampling_rate"])

In [49]:
from transformers import pipeline
zero_shot_classifier = pipeline(
    task="zero-shot-audio-classification",
    model="laion/clap-htsat-unfused")

In [54]:
print(zero_shot_classifier.feature_extractor.sampling_rate)
audio_sample['audio']['sampling_rate']

48000


48000

In [55]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=48000))

In [58]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=48000))
candidate_labels = ["Sound of a dogukan", "Sound of a cat", "Sound of a bird", "Sound of a vehicle", "Sound of a human"]
zero_shot_classifier(audio_sample["audio"]['array'],
                     candidate_labels=candidate_labels)

[{'score': 0.7864915728569031, 'label': 'Sound of a vehicle'},
 {'score': 0.17673511803150177, 'label': 'Sound of a human'},
 {'score': 0.01859550178050995, 'label': 'Sound of a bird'},
 {'score': 0.010697100311517715, 'label': 'Sound of a dogukan'},
 {'score': 0.007480670232325792, 'label': 'Sound of a cat'}]